#用阈值创建二元特征

在前一个主题，我们介绍了数据转换成标准正态分布的方法。现在，我们看看另一种完全不同的转换方法。

当不需要呈标准化分布的数据时，我们可以不处理它们直接使用；但是，如果有足够理由，直接使用也许是聪明的做法。通常，尤其是处理连续数据时，可以通过建立二元特征来分割数据。

<!-- TEASER_END -->

##Getting ready

通常建立二元特征是非常有用的方法，不过要格外小心。我们还是用`boston`数据集来学习如何创建二元特征。

首先，加载`boston`数据集：

In [1]:
from sklearn import datasets
boston = datasets.load_boston()
import numpy as np

##How to do it...

与标准化处理类似，scikit-learn有两种方法二元特征：

- `preprocessing.binarize`（一个函数）
- `preprocessing.Binarizer`（一个类）

`boston`数据集的因变量是房子的价格中位数（单位：千美元）。这个数据集适合测试回归和其他连续型预测算法，但是假如现在我们想预测一座房子的价格是否高于总体均值。要解决这个问题，我们需要创建一个均值的阈值。如果一个值比均值大，则为`1`；否则，则为`0`：

In [6]:
from sklearn import preprocessing
new_target = preprocessing.binarize(boston.target, threshold=boston.target.mean())
new_target[0,:5]

array([ 1.,  0.,  1.,  1.,  1.])

很容易，让我们检查一下：

In [22]:
(boston.target[:5] > boston.target.mean()).astype(int)

array([1, 0, 1, 1, 1])

既然Numpy已经很简单了，为什么还要用scikit-learn的函数呢？管道命令，将在*用管道命令联接多个预处理步骤*一节中介绍，会解释这个问题；要用管道命令就要用`Binarizer`类：

In [24]:
bin = preprocessing.Binarizer(boston.target.mean())
new_target = bin.fit_transform(boston.target)
new_target[0,:5]

array([ 1.,  0.,  1.,  1.,  1.])

##How it works...

方法看着非常简单；其实scikit-learn在底层创建一个检测层，如果被监测的值比阈值大就返回`Ture`。然后把满足条件的值更新为`1`，不满足条件的更新为`0`。

##There's more...

让我们再介绍一些稀疏矩阵和`fit`方法的知识。

###稀疏矩阵

稀疏矩阵的`0`是不被存储的；这样可以节省很多空间。这就为`binarizer`造成了问题，需要指定阈值参数`threshold`不小于`0`来解决，如果`threshold`小于`0`就会出现错误：

In [31]:
from scipy.sparse import coo
spar = coo.coo_matrix(np.random.binomial(1, .25, 100))
preprocessing.binarize(spar, threshold=-1)

ValueError: Cannot binarize a sparse matrix with threshold < 0

###`fit`方法

`binarizer`类里面有`fit`方法，但是它只是通用接口，并没有实际的拟合操作，仅返回对象。